In [85]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from libs import *

In [86]:
model_name = 'google/flan-t5-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=19)

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at google/flan-t5-base and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [87]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

T5ForSequenceClassification(
  (transformer): T5Model(
    (shared): Embedding(32128, 768)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 768)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=768, out_features=768, bias=False)
                (k): Linear(in_features=768, out_features=768, bias=False)
                (v): Linear(in_features=768, out_features=768, bias=False)
                (o): Linear(in_features=768, out_features=768, bias=False)
                (relative_attention_bias): Embedding(32, 12)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseGatedActDense(
                (wi_0): Linear(in_features=768, out_features=2048, bias=False)
                (wi_1): Linea

In [88]:
def compute_top_k_accuracy(preds, labels, k=1):
    top_k_preds = np.argsort(preds, axis=1)[:, -k:]
    top_k_accuracy = np.any(top_k_preds == np.expand_dims(labels, axis=1), axis=1).mean()
    return top_k_accuracy

def preprocess_function(examples, tokenizer):
    return tokenizer(examples["Question"], truncation=True, padding = 'max_length', max_length=512)   

In [89]:
seed = 42
df_test_asdiv = pd.read_csv(f'data_first_ver/{seed}_test_set_asdiv.csv')
df_test_mcas = pd.read_csv(f'data_first_ver/{seed}_test_set_mcas.csv')

dataset_test_asdiv = Dataset.from_pandas(df_test_asdiv)
dataset_test_mcas = Dataset.from_pandas(df_test_mcas)

tokenized_dataset_test_asdiv = dataset_test_asdiv.map(lambda x: preprocess_function(x, tokenizer), batched=True)
tokenized_dataset_test_mcas = dataset_test_mcas.map(lambda x: preprocess_function(x, tokenizer), batched=True)

Map:   0%|          | 0/62 [00:00<?, ? examples/s]

Map:   0%|          | 0/73 [00:00<?, ? examples/s]

In [90]:
trainer = Trainer(
    model=model,
    args=TrainingArguments(
        per_device_eval_batch_size= 1,
        output_dir= './result'
    ),
    tokenizer=tokenizer,
    data_collator=None
)

print(f"Evaluation on test set for seed {seed}...")
test_results_asdiv = trainer.evaluate(eval_dataset=tokenized_dataset_test_asdiv)
test_results_mcas = trainer.evaluate(eval_dataset=tokenized_dataset_test_mcas)
print(f'ASDIV: {test_results_asdiv}')
print(f'MCAS: {test_results_mcas}')

c:\Users\lehai\anaconda3\envs\temasek_genAI\lib\site-packages\accelerate\accelerator.py:444: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Evaluation on test set for seed 42...


  0%|          | 0/62 [00:00<?, ?it/s]

  0%|          | 0/73 [00:00<?, ?it/s]

ASDIV: {'eval_loss': 2.9034464359283447, 'eval_runtime': 92.4269, 'eval_samples_per_second': 0.671, 'eval_steps_per_second': 0.671}
MCAS: {'eval_loss': 2.99104380607605, 'eval_runtime': 106.7801, 'eval_samples_per_second': 0.684, 'eval_steps_per_second': 0.684}


In [91]:
preds_asdiv = trainer.predict(tokenized_dataset_test_asdiv).predictions
labels_asdiv = np.array(tokenized_dataset_test_asdiv["label"])

preds_mcas = trainer.predict(tokenized_dataset_test_mcas).predictions
labels_mcas = np.array(tokenized_dataset_test_mcas["label"])

  0%|          | 0/62 [00:00<?, ?it/s]

  0%|          | 0/73 [00:00<?, ?it/s]

In [100]:
labels_asdiv.shape

(62,)

In [108]:
preds_asdiv[1].shape

(62, 512, 768)

In [114]:
np.array(preds_asdiv[0]).shape

(62, 19)

In [ ]:
labels_asdiv.shape

(62,)

In [ ]:
np.expand_dims(labels_asdiv, axis=1) # Change to shape (62,1)

array([[ 5],
       [ 3],
       [15],
       [ 2],
       [18],
       [ 4],
       [ 4],
       [ 4],
       [ 6],
       [ 3],
       [14],
       [13],
       [10],
       [16],
       [16],
       [14],
       [12],
       [ 3],
       [ 3],
       [18],
       [ 4],
       [17],
       [ 3],
       [ 1],
       [12],
       [ 4],
       [ 7],
       [ 7],
       [ 4],
       [ 3],
       [13],
       [ 3],
       [ 4],
       [ 9],
       [ 0],
       [ 2],
       [10],
       [14],
       [10],
       [14],
       [ 2],
       [ 5],
       [ 3],
       [ 4],
       [10],
       [ 0],
       [ 7],
       [18],
       [ 3],
       [ 7],
       [13],
       [18],
       [ 3],
       [11],
       [ 8],
       [ 0],
       [ 0],
       [ 3],
       [10],
       [12],
       [ 0],
       [15]])

In [109]:
def get_top_k_predictions(predictions, k=1):
    top_k_preds = np.argsort(predictions, axis=1)[:, -k:]
    return top_k_preds

def compute_top_k_accuracy(preds, labels, k=1):
    top_k_preds = np.argsort(preds, axis=1)[:, -k:]
    top_k_accuracy = np.any(top_k_preds == np.expand_dims(labels, axis=1), axis=1).mean()
    print(np.expand_dims(labels, axis=1))
    return top_k_accuracy


compute_top_k_accuracy(preds_asdiv, labels_asdiv, k=1)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (2, 62) + inhomogeneous part.

In [ ]:
compute_top_k_accuracy(preds_asdiv, labels_asdiv, k=18)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (2, 62) + inhomogeneous part.

In [ ]:
for k in range(1, 4):
        top_k_accuracies_asdiv[k] += compute_top_k_accuracy(preds_asdiv, labels_asdiv, k=k)
        top_k_accuracies_mcas[k] += compute_top_k_accuracy(preds_mcas, labels_mcas, k=k)

results = []
results.append([f"Seed {seed}", test_results_asdiv['eval_accuracy'], test_results_mcas['eval_accuracy']])
test_acc_asdiv += test_results_asdiv['eval_accuracy']
test_acc_mcas += test_results_mcas['eval_accuracy']